# Test Gemini vs Ensemble for MMLU


to do:

-ignore claude, add gemma, palm

In [1]:
from pprint import pprint
from datetime import datetime
from datasets import load_dataset

from utils import Annotate

In [2]:
seed = 42
now = datetime.now().strftime("%Y%m%d")

In [3]:
dataset = load_dataset("cais/mmlu", "all")

# # take a small sample for dev purposes
dataset = dataset['test'].shuffle(seed=seed).select(range(5))

# user provided data description
DESCRIPTION = """
This is a massive multitask test consisting of multiple-choice questions from various branches of knowledge.
The test spans subjects in the humanities, social sciences, hard sciences, and other areas that are important for some people to learn.
To attain high accuracy on this test, models must possess extensive world knowledge and problem solving ability.
This covers 57 subjects  across STEM, the humanities, the social sciences, and more. 
It ranges in difficulty from an elementary level to an advanced professional level, and it tests both world knowledge and problem solving ability. 
Subjects range from traditional areas, such as mathematics and history, to more specialized areas like law and ethics.
"""


In [4]:
gemini_prompt_template = """
<QUESTION>
{datapoint}
</QUESTION>
------------

<CHOICES>
{labels}
</choices>
------------

INSTRUCTION:
- read the above question carefully.
- you are given 4 choices seperated by comma in <CHOICES>.
- take your time and pick the precise correct answer from <CHOICES> for the given <QUESTION>.
- remember that there is always only one correct answer.
- return the exact correct answer from <CHOICES>. Don't provide explanations.
"""

In [5]:
prompt = [gemini_prompt_template.format(description= DESCRIPTION,
                                        datapoint=x['question'],
                                        labels=x['choices']) for x in dataset]
print(len(prompt))

5


In [8]:
models = [
    "palm",
    # "gemini",
    # "claude"
    ]

ann = Annotate()


In [16]:
output_dict = await ann.classification(prompt, models=models)

Creating tasks: 10it [00:00, 44104.14it/s]           
Gathering palm results: 100%|██████████| 5/5 [00:02<00:00,  1.70it/s]


In [17]:
output_dict

{'palm': [" '12.2 eV'",
  ' left submental lymph node.',
  ' The sample sizes are different.',
  ' lieutenant general',
  ' the need to construct monumental works, such as step-pyramids and temples']}

In [33]:
llm_response = {}

for m in models:   
    llm_response[m] = [dataset['choices'][idx].index(r.strip().strip("'")) for idx, r in enumerate(output_dict[m])]
llm_response

{'palm': [2, 0, 0, 2, 1]}

#  GLAD

In [34]:
from utils import glad

In [43]:
num_labels = sum(len(lst) for lst in llm_response.values())
num_labelers =  len(list(llm_response.values())[0])
num_tasks = len(llm_response)
num_classes = 4
z  = 1/num_classes


tc = [num_labels, num_labelers, num_tasks, num_classes]
tc.extend([z] * tc[-1])

tc

[5, 5, 1, 4, 0.25, 0.25, 0.25, 0.25]

In [44]:
# import json

# with open('./data/output/annotation_output__20240515.json', 'r') as file:
#     ddddd = json.load(file)

In [45]:
glad(llm_response, tc)

IndexError: index 1 is out of bounds for axis 0 with size 1